In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Download NLTK resources (do it once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
file=r"C:\Users\jayen\Text-sentiment-analysis-general-purpose\artifacts\Emotions.csv"

df=pd.read_csv(file)
df

,text,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear
...,...,...
295157,im having ssa examination tomorrow in the morn...,sadness
295158,i constantly worry about their fight against n...,joy
295159,i feel its important to share this info for th...,joy
295160,i truly feel that if you are passionate enough...,joy


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295162 entries, 0 to 295161
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   text     295162 non-null  object
 1   Emotion  295162 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


In [35]:
df.isnull().sum()

text       0
Emotion    0
dtype: int64

In [17]:
df['Emotion'].value_counts().head(50)

Emotion
neutral                     64574
sadness                     20036
joy                         19147
anger                       12455
love                        11903
approval                    11259
admiration                  10531
worry                        8459
annoyance                    8342
fear                         8249
disapproval                  7686
gratitude                    7075
surprise                     6378
amusement                    6130
curiosity                    5885
happiness                    5209
confusion                    4938
realization                  4714
disappointment               4706
optimism                     4519
caring                       3523
example_very_unclear         3411
excitement                   3020
disgust                      2914
relief                       2314
desire                       2147
fun                          1776
remorse                      1510
embarrassment                1433
hate  

In [36]:

threshold_value = 2914
#df['Emotion'] = pd.to_numeric(df['Emotion'], errors='coerce')
# Filter out rows where the "Emotions" column is below the threshold for the label "pride"
# Filter out rows where the "Emotions" column is below the threshold for the label "pride"
l=df['Emotion'].value_counts()
print(l)
# Filter out labels that occur more than threshold_count times
# Filter out labels that occur more than threshold_count times
selected_labels = l[l >= threshold_value].index
print(selected_labels)


Emotion
neutral                                                         64574
sadness                                                         20036
joy                                                             19147
anger                                                           12455
love                                                            11903
                                                                ...  
curiosity,gratitude,nervousness                                     1
anger,confusion,disappointment,disgust,embarrassment,sadness        1
admiration,amusement,desire,love                                    1
amusement,disapproval,love                                          1
disgust,love,sadness                                                1
Name: count, Length: 2018, dtype: int64
Index(['neutral', 'sadness', 'joy', 'anger', 'love', 'approval', 'admiration',
       'worry', 'annoyance', 'fear', 'disapproval', 'gratitude', 'surprise',
       'amusement', 'curio

In [37]:
# Filter out rows where the "Emotions" column is below the threshold for the selected labels
selected_labels = df['Emotion'].value_counts()[df['Emotion'].value_counts() >= threshold_value].index

# Filter out rows where the "Emotions" column is below the threshold for the selected labels
f_df = df[df['Emotion'].isin(selected_labels)]



In [38]:
f_df['Emotion'].value_counts()

Emotion
neutral                 64574
sadness                 20036
joy                     19147
anger                   12455
love                    11903
approval                11259
admiration              10531
worry                    8459
annoyance                8342
fear                     8249
disapproval              7686
gratitude                7075
surprise                 6378
amusement                6130
curiosity                5885
happiness                5209
confusion                4938
realization              4714
disappointment           4706
optimism                 4519
caring                   3523
example_very_unclear     3411
excitement               3020
disgust                  2914
Name: count, dtype: int64

In [41]:
#NOW ILL DO SOME PREPROCESSING STEPS FOR THAT ILL CREATE A FUNCTION
def preprocessing(q):
 
  q=str(q).lower().strip()#strip gives a string in return if u use split it will give a list of words
  #The strip() method in Python is used to remove leading and trailing characters (whitespace characters by default) from a string before " Hello, world! " after"Hello, world!"
   # Replace certain special characters with their string equivalents
  q = q.replace('%', 'percent')
  q = q.replace('$', 'dollar ')
  q = q.replace('₹', 'rupee ')
  q = q.replace('€', 'euro ')
  q = q.replace('@', 'at')
  q=q.replace('&','and')

  q = re.sub(r'([0-9]+)000000000', r'\1b', re.sub(r'([0-9]+)000000', r'\1m', re.sub(r'([0-9]+)000', r'\1k', q)))#Input: "1500000000 Output: "1.5b" it will do this
  q=re.sub(r'\b(?:\d{1,4}[/-]\d{1,2}[/-]\d{1,4}|\d{1,2}[/-]\d{1,2}[/-]\d{2,4})\b','',q)#for removing dates
  q=re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b','',q)#removing email
  q= re.sub(r'https?://\S+|www\.\S+','',q)#removing urls



    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
  contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
  q_decontracted=[]

  q_decontracted = [contractions[i] if i in contractions else i for i in q.split()]


  #now i join this word into q
  q=' '.join(q_decontracted)
  q = q.replace("'ve", " have")
  q = q.replace("n't", " not")
  q = q.replace("'re", " are")
  q = q.replace("'ll", " will")
  # Removing HTML tags
  q = BeautifulSoup(q)
  q = q.get_text()
  # Remove punctuations like ,.""[]
  q =re.sub(r'[^\w\s{}\/<>\'"\-_=+()*~:;]', ' ', q)

  q = word_tokenize(q)  # Convert to lowercase

  # Remove stopwords and non-alphabetic characters, perform lemmatization
  q = [
        lemmatizer.lemmatize(word) for word in q
        if word not in stop_words
    ]



  return ' '.join(q)#ALWAYS REMEMBER IN ORDER TO USE WORD2VEC UR SENTENSES SHOULD BE IN TOKKENS OR WORDS BUT IN THIS CASE WE HAVE TO TRAIN OUR MODEL WE HAVE TO TOOKENIZE
  #THE WORDS SO WE USE JOIN THEN AFTER WE TOKKENIZE IT WONT BE ABLE TO CONCATINATE LIST OF STRINGS IMPORTANT. contain lists instead of strings.





In [42]:
f_df['preprocessed']=f_df['text'].apply(preprocessing)

C:\Users\jayen\AppData\Local\Temp\ipykernel_9944\1991588443.py:154: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  q = BeautifulSoup(q)
C:\Users\jayen\AppData\Local\Temp\ipykernel_9944\2005927719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_df['preprocessed']=f_df['text'].apply(preprocessing)


In [1]:
pip install NeatText


     ---------------------------------------- 0.0/114.7 kB ? eta -:--:--
     --- ------------------------------------ 10.2/114.7 kB ? eta -:--:--
     ------ ------------------------------ 20.5/114.7 kB 108.9 kB/s eta 0:00:01
     ------------------- ----------------- 61.4/114.7 kB 326.1 kB/s eta 0:00:01
     ------------------------------------ 114.7/114.7 kB 512.2 kB/s eta 0:00:00


In [51]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame with the "Emotions" column
# df = ...

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the "Emotions" column
f_df['Numerical_Emo'] = label_encoder.fit_transform(f_df['Emotion'])

C:\Users\jayen\AppData\Local\Temp\ipykernel_9944\1487351306.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_df['Numerical_Emo'] = label_encoder.fit_transform(f_df['Emotion'])


In [61]:
from sklearn.feature_extraction.text import CountVectorizer
# Create a CountVectorizer instance
vectorizer = CountVectorizer()

x=np.array(f_df['preprocessed'])
y=np.array(f_df['Numerical_Emo'])

X=vectorizer.fit_transform(x)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [64]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.Collecting xgboost
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/99.8 MB 3.2 MB/s eta 0:00:32
   ---------------------------------------- 0.8/99.8 MB 5.4 MB/s eta 0:00:19
   ---------------------------------------- 0.8/99.8 MB 4.5 MB/s eta 0:00:23
   ---------------------------------------- 1.0/99.8 MB 4.6 MB/s eta 0:00:22
   ---------------------------------------- 1.0/99.8 MB 4.1 MB/s eta 0:00:25
    --------------------------------------- 1.4/99.8 MB 4.1 MB/s eta 0:00:24
    --------------------------------------- 1.6/99.8 MB 4.2 MB/s eta 0:00:24
    --------------------------------------- 1.6/99.8 MB 4.2 MB/s eta 0:00:24
    --------------------------------------- 1.7/99.8 MB 3.8 MB/s eta 0:00:26
    --------------------------------------- 1.9/99.8 MB 3.5 MB/s eta 0:00:28
    --

In [65]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
v=xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y)))#objective='multi:softmax' indicates that the model should perform multiclass classification, and num_class=len(set(y)) sets the number of classes based on the unique labels in your target variable (y).
v.fit(X_train,y_train)
# Calculate accuracy
y_pred=v.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of XgboostClassifier: {accuracy * 100:.2f}%")

Accuracy of XgboostClassifier: 49.86%


: 

In [30]:
#WE CAN USE THIS NEAT TEXT TOO
import neattext as nt
def clean_text(text):
    docx = nt.TextFrame(text)
    cleaned_text = docx.remove_puncts()
    cleaned_text = cleaned_text.remove_special_characters()
    cleaned_text = cleaned_text.remove_userhandles()
    cleaned_text = cleaned_text.remove_urls()

    return cleaned_text

text="@user 575864 23/2/2000 jayendra @$ .{}[] !` jay@gamil.com"
s=clean_text(text)
print(s)


user 575864 2322000 jayendra    jaygamilcom


In [21]:
import neattext.functions as nt
dir(nt)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [96]:
f_df.sample(10)

,text,Emotion,preprocessed,Numerical_Emo
183915,I wish. Its too tiny to see from up here.,neutral,wish tiny see,29
273453,@emjhoistarr Hello. New follower lang po. Twee...,neutral,atemjhoistarr hello new follower lang po tweet...,29
47297,"I lovee mazzy star sm, Flowers in December is ...",love,lovee mazzy star sm flower december favorite s...,27
61414,He also says something about waking up from a ...,anger,also say something waking nap gameplay reveal ...,4
2602,i feel relieved that a rescue party has arrived,joy,feel relieved rescue party arrived,26
268191,@merchantships You're welcome! Math is really...,neutral,atmerchantships welcome math really * * favori...,29
269694,@megan_ward i am,neutral,atmegan_ward,29
97346,"[NAME] damnit, have your upvote",anger,name damnit upvote,4
103267,I think [NAME] is starting to get too much bla...,neutral,think name starting get much blame everybody r...,29
55652,Actually there are some terminals now that let...,sadness,actually terminal let disable lmao sad,35


In [52]:
f_df['Emotion'].value_counts()

Emotion
neutral                 64574
sadness                 20036
joy                     19147
anger                   12455
love                    11903
approval                11259
admiration              10531
worry                    8459
annoyance                8342
fear                     8249
disapproval              7686
gratitude                7075
surprise                 6378
amusement                6130
curiosity                5885
happiness                5209
confusion                4938
realization              4714
disappointment           4706
optimism                 4519
caring                   3523
example_very_unclear     3411
excitement               3020
disgust                  2914
Name: count, dtype: int64

In [53]:
f_df['Numerical_Emo'].value_counts()

Numerical_Emo
18    64574
21    20036
16    19147
2     12455
17    11903
4     11259
0     10531
23     8459
3      8342
13     8249
9      7686
14     7075
22     6378
1      6130
7      5885
15     5209
6      4938
20     4714
8      4706
19     4519
5      3523
11     3411
12     3020
10     2914
Name: count, dtype: int64

In [59]:
f_df

,text,Emotion,preprocessed,Numerical_Emo
0,i seriously hate one subject to death but now ...,fear,seriously hate one subject death feel reluctan...,13
1,im so full of life i feel appalled,anger,im full life feel appalled,2
2,i sit here to write i start to dig out my feel...,fear,sit write start dig feeling think afraid accep...,13
3,ive been really angry with r and i feel like a...,joy,ive really angry r feel like idiot trusting fi...,16
4,i feel suspicious if there is no one outside l...,fear,feel suspicious one outside like rapture happe...,13
...,...,...,...,...
295157,im having ssa examination tomorrow in the morn...,sadness,im ssa examination tomorrow morning im quite w...,21
295158,i constantly worry about their fight against n...,joy,constantly worry fight nature push limit inner...,16
295159,i feel its important to share this info for th...,joy,feel important share info experience thing,16
295160,i truly feel that if you are passionate enough...,joy,truly feel passionate enough something stay tr...,16


In [43]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [44]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(f_df['preprocessed'])
sequences = tokenizer.texts_to_sequences(f_df['preprocessed'])

In [45]:
max_length = 200  # Define your maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length)#THIS WILL BE MY ACTUAL DATA THAT'LL TRAIN TEST AND SPLIT

In [14]:
padded_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [46]:
word_index = tokenizer.word_index#THESE ARE MY UNIQUE WORDS WITH ITS UNIQUE INDEX
print(word_index)

{'feel': 1, 'name': 2, 'like': 3, 'feeling': 4, 'get': 5, 'would': 6, 'one': 7, 'know': 8, 'time': 9, 'people': 10, 'love': 11, 'good': 12, 'really': 13, 'im': 14, 'think': 15, 'make': 16, 'day': 17, 'go': 18, 'thing': 19, 'got': 20, 'see': 21, 'want': 22, 'still': 23, 'even': 24, 'much': 25, 'look': 26, 'going': 27, 'u': 28, 'way': 29, 'year': 30, 'well': 31, 'right': 32, 'lol': 33, 'need': 34, 'back': 35, 'could': 36, 'work': 37, 'say': 38, 'oh': 39, "'s": 40, 'never': 41, 'thanks': 42, 'guy': 43, 'bad': 44, 'life': 45, 'something': 46, 'also': 47, 'little': 48, "''": 49, 'happy': 50, 'yeah': 51, 'pretty': 52, 'someone': 53, 'better': 54, 'though': 55, 'thought': 56, 'man': 57, 'sure': 58, 'great': 59, 'take': 60, 'friend': 61, 'game': 62, 'always': 63, 'thank': 64, 'actually': 65, 'hope': 66, 'mean': 67, 'new': 68, 'today': 69, 'last': 70, 'come': 71, 'first': 72, 'made': 73, 'dont': 74, 'best': 75, 'getting': 76, 'lot': 77, 'let': 78, 'ever': 79, 'said': 80, 'post': 81, '2': 82, 'b

In [47]:
num_words = len(word_index)+1

In [48]:
## Creating model
from tensorflow.keras.layers import Dropout
embedding_vector_features=300 ##features representation each word wil get convert into a vector of 300
model=Sequential()
model.add(Embedding(num_words,embedding_vector_features,input_length=max_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(39,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 300)          18084000  
                                                                 
 dropout (Dropout)           (None, 200, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 39)                3939      
                                                                 
Total params: 18248339 (69.61 MB)
Trainable params: 18248339 (69.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [54]:
import numpy as np
X_final=np.array(padded_sequences)
y_final=np.array(f_df['Numerical_Emo'])

In [55]:
X_final.shape,y_final.shape

((245063, 200), (245063,))

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)

In [57]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', patience=3, mode='max', verbose=1, baseline=0.95)

In [58]:
model.fit(X_final,y_final, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
 244/6127 [>.............................] - ETA: 38:46 - loss: 2.8313 - accuracy: 0.2866

KeyboardInterrupt: 

In [1]:
import os
artifacts_folder = "models"
os.makedirs(artifacts_folder, exist_ok=True)
# Save the trained model to the "artifacts" folder
model_filename = os.path.join(artifacts_folder, 'Emotions_model')
model.save(model_filename)

NameError: name 'model' is not defined